# Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto

# Gokhan Ince

## PART 1

### For this assignment, we are required to explore and cluster the neighborhoods in Toronto.

### First, we need to import pandas library to pull data from the link. And then assign the URL to a variable. And then pull the data by using read_html function to put them in a dataframe.

In [ ]:
import pandas as pd 

# I assigned the link to url variable
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

# I creaated a dataframe by pulling data from the link
df = pd.read_html(url, header = 0)[0]

### After that we need to ignore the cells which has no borough information.

In [ ]:
#Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
df = df[df.Borough != 'Not assigned']

### Sometimes a postal code is related to different neighbourhoods. We need to combine them.

In [ ]:
#More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.
df = df.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(list).apply(lambda x:', '.join(x)).to_frame().reset_index()

### If a cell has borough info but not assigned to a neighborhood, the neighborhood will be the same as the borough.

In [ ]:
#If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.
for index, row in df.iterrows():
    if row['Neighbourhood'] == 'Not assigned':
        row['Neighbourhood'] = row['Borough']

### Now, our dataframe is ready. Let's see its shape.

In [ ]:
#shape of the dataframe
df.shape

### Its shape is (103,3). It means our dataframe has 3 columns and 103 rows.
### Now we can see our data

In [ ]:
# data
df

## Part 2

### Now, I will download the geospatial data of Toronto, create a dataframe and then show some data from the dataframe.

In [ ]:
!wget -q -O 'Toronto_long_lat_data.csv'  http://cocl.us/Geospatial_data
df_lon_lat = pd.read_csv('Toronto_long_lat_data.csv')
df_lon_lat.head()

### I will rename the column names to make it more meaningful and it will look better.

In [ ]:
df_lon_lat.columns=['Postcode','Latitude','Longitude']
df_lon_lat.head()

### Now I will merge the dataframes by using Postcode. It will update my df_toronto dataframe.

In [ ]:
df_toronto = pd.merge(df,df_lon_lat[['Postcode','Latitude', 'Longitude']], on='Postcode')
df_toronto

## Part 3

### Now, I will import a library to convert an address in to latitude and longitude values.

In [ ]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
# Matplotlib and associated modules
import matplotlib.cm as cm
import matplotlib.colors as colors

### Now, I will detect the latitude and longitude values of Toronto.

In [ ]:
# Toronto is assigned to the variable "address"
address = 'Toronto, ON'
geolocator = Nominatim(user_agent="Toronto")
location = geolocator.geocode(address)
latitude_toronto = location.latitude
longitude_toronto = location.longitude
print('Latitude of Toronto is {} and longtitude is {}.'.format(latitude_toronto, longitude_toronto))

### Now, I will import Folium library to generate maps.

In [ ]:
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

### The library is ready. Now we can generate our map.

In [ ]:
map_toronto = folium.Map(location=[latitude_toronto, longitude_toronto], zoom_start=10)
# added markers to map
for lat, lng, borough, Neighbourhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighbourhood']):
    label = '{}, {}'.format(Neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    


In [ ]:
# At last we are able to see our map.

In [ ]:
map_toronto

## Thanks for reviewing
## Regards
## Gokhan Ince